## 계산 그래프
- 복잡한 수학적 표현식은 계산 그래프 (Computational Graphs)로 시각화할 수 있다. 
- 예를 들어, 다음과 같이 덧셈이 2개와 곱셈 1개로 이루어진 식이 있다고 할 때,

$$ e=(a+b)\times(b+1) $$

- 곱셈은 $e=c \times d$로, 덧셈은 $c=a+b$, $d=b+1$로 나타낼 수 있으므로 계산 그래프로 나타내면 다음과 같다.

<img src="https://cdn-ak.f.st-hatena.com/images/fotolife/y/yusuke_ujitoko/20161231/20161231102417.png" width="400"></img>

- 위 식을 값을 구하는 것은 입력 변수에 특정값을 할당하여 순서대로 계산하는 것이다. 
- 예를 들어, $a=2$, $b=3$을 더하면, $e=20$이 된다. 

<img src="https://cdn-ak.f.st-hatena.com/images/fotolife/y/yusuke_ujitoko/20161231/20161231102756.png" width="400"></img>

- 이와 같이 계산 그래프를 사용하면 아무리 복잡한 계산도 각 노드별로 지역 계산으로 단순화할 수 있다.

## 순전파와 역전파
- 계산 결과를 왼쪽에서 오른쪽으로 진행하는 전달하는 것을 순전파(forward propagation)라고 한다.
- 오른쪽에서 왼쪽으로 전달하는 것을 역전파(backward propagation)라고 하고 이는 연쇄법칙(chain rule)에 의해 전달된다.  
<img src="https://kratzert.github.io/images/bn_backpass/chainrule_example.PNG" width="600"></img>

- 연쇄법칙은 미분 공식에서 가장 중요한 것 중의 하나로 합성 함수의 미분은 각 함수의 곱으로 주어진다는 것이다.

> $f$와 $g$가 미분 가능하고, $ F = f \circ g $가 $F(x) = f(g(x))$로 정의된 합성함수라면, $F$는 $x$에서 미분 가능하고, $F'$는 다음과 같은 곱으로 주어진다. 

$$ F'(x) = f'(g(x)) \cdot g'(x) $$

> 이를 라이프니츠 기호로 나타내면, $y = f(u) $ 와 $ u = g(x) $가 모두 미분가능한 함수일 때 다음이 성립한다. 

$$ \frac{dy}{dx} = \frac{dy}{du}\frac{du}{dx} $$

- 따라서 연쇄법치게 따라, 다음과 같이 $x$에 대한 $L$의 미분($x$가 증가했을 때, $L$의 변화율) 은 $L$에 대한 $z$의 미분과 $z$에 대한 $x$의 미분의 곱으로 나타낼 수 있다. 

$$\frac{\partial{L}}{\partial{x}} = \frac{\partial{L}}{\partial{z}}\frac{\partial{z}}{\partial{x}} $$

# 활성화 함수
### ReLu 
- ReLU는 다음과 같다

$$
y =\begin{cases} 
    x & (x > 0) \\ 
    0 & (x \leq 0)\\
   \end{cases}
$$

- $x$에 대한 $y$의 미분은 다음과 같다. (순전파의 $x$값이 0보다 크면 그대로 전달하고, 0 이하면 전달하지 않는다.)
$$
\frac{\partial{y}}{\partial{x}} = \begin{cases} 
    1 & (x > 0) \\ 
    0 & (x \leq 0)\\
   \end{cases}
$$

- 구현 예제이다.

In [1]:
class Relu:
    def __init__(self):
        self.mask = None # 요소의 값이 0보다 작거나 같으면 True, 아니면 False로 셋팅하는 마스크이다.
        
    def forward(self, x):
        self.mask = (x <= 0) 
        out = x.copy() 
        out[self.mask] = 0 # 
        
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        
        return dx

### Sigmoid
- 시그모이드는 다음과 같다.
$$ y = \frac{1}{(1 + e^{-x})} $$

- $y = \frac{1}{u}$ 이고, $ u = 1 + e^{-x} $라고 하면, $f'(u) = - \frac{1}{u^2}$이고, $ g'(x) = (-1) \cdot e^{-x}$이므로,

- 시그모이드 함수의 미분은 다음과 같이 전개할 수 있다.

$$
\begin{split}
F'(x) & = - \frac{1}{{\left(1 + e^{-x}\right)}^2} \cdot -e^{-x} \\
& = \frac{e^{-x}}{{\left(1 + e^{-x}\right)}^2} \\
& = \frac{e^{-x} + 1 - 1}{{\left(1 + e^{-x}\right)}^2} \\
& = \frac{1 + e^{-x}}{{\left(1 + e^{-x}\right)}^2} - {\left(\frac{1}{1 + e^{-x}}\right)}^2 \\
& = \frac{1}{1 + e^{-x}} - {\left(\frac{1}{1 + e^{-x}}\right)}^2 \\
& = y - y^2 \\
& = y(1-y)
\end{split}
$$

- 즉, 시그모이드의 역전파는 출력만으로 계산할 수 있다.
- 구현 예는 다음과 같다.

In [2]:
class Sigmoid:
    def __init__(self):
        self.out = None
        
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out
    
    def backward(self, out):
        dx = dout * (1.0 - self.out) * self.out
        return dx

## Affine/Softmax
- 순전파 때 수행하는 벡터의 내적을 기하학에서는 **아핀 변환(affine transformation)**이라고 한다.

- 벡터 내적 $\textbf{X} \bullet \textbf{W}$에 역전파(미분)은 다음과 같다. ([벡터 미분과 행렬 미분](http://darkpgmr.tistory.com/141))
$$ \frac{\partial{L}}{\partial{\textbf{X}}} = \frac{\partial{L}}{\partial{\textbf{Y}}} \cdot \textbf{W}^{\top}  $$
$$ \frac{\partial{L}}{\partial{\textbf{W}}} = \textbf{X}^{\top} \cdot \frac{\partial{L}}{\partial{\textbf{Y}}}   $$